In [2]:
from IPython.core.display import display, HTML
from scipy.spatial.transform import Rotation
import h5py
from matplotlib import pyplot as plt
import numpy as np
from pytorch3dunet.unet3d.losses import *
from pytorch3dunet.unet3d.metrics import MeanIoU
from pathlib import Path
import h5py
import numpy as np
import prody
from potsim2 import PotGrid
import sklearn
from sklearn.metrics import matthews_corrcoef as mcc 
from sklearn.metrics import f1_score
import os
import pandas as pd
import shutil
from openbabel import openbabel

/var/folders/y5/6tr71z916z32tv84d2pvh_4jrmvb0w/T/ipykernel_74787/2511895280.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/Users/tevfik/anaconda3/envs/cavity/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


SRC_DIR = '/Users/tevfik/Sandbox/github/PHD/data/src_data/pdbbind/refined-set'
DEST_DIR = '/Users/tevfik/Sandbox/github/PHD/data/dest_data'
PROTEIN_NAME = '1a1e'

srcPdbFile = f"{SRC_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_protein.pdb"
srcMolFile = f"{SRC_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_protein.mol2"
destPdbFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}.pdb"
destLigandPdbFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_ligand.pdb"
destSelectedPdbFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_selected.pdb"
destCalculatedPocketFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_pocket.pdb"

# Convert Protein to pdb.
if os.path.exists(srcPdbFile):
    print(f"Found existing protein pdb file in {srcPdbFile}. Copying over to {destPdbFile}")
    shutil.copyfile(srcPdbFile, destPdbFile)
elif os.path.exists(srcMolFile):
    print(f"Found existing protein MOL file in {srcMolFile}. Converting to {destPdbFile}")
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("mol2", "pdb")
    # convert protein to pdb format from mol2 format
    protein = openbabel.OBMol()
    obConversion.ReadFile(protein, srcMolFile)
    obConversion.WriteFile(protein, destPdbFile)

Found existing protein pdb file in /Users/tevfik/Sandbox/github/PHD/data/src_data/pdbbind/refined-set/1a1e/1a1e_protein.pdb. Copying over to /Users/tevfik/Sandbox/github/PHD/data/dest_data/1a1e/1a1e.pdb


In [4]:
# Using `prody` library remove water molecules from pdb file content
structure = prody.parsePDB(destPdbFile)
structure = structure.select('protein')
prody.writePDB(destPdbFile, structure)

# pdb2pqr fails to read pdbs with the one line header generated by ProDy...
with open(destPdbFile, 'r') as fin:
    data = fin.read().splitlines(True)
with open(destPdbFile, 'w') as fout:
    fout.writelines(data[1:])

@> 3279 atoms and 1 coordinate set(s) were parsed in 0.06s.


In [5]:
# convert ligand to pdb
ligand = openbabel.OBMol()
srcMolFile1 = f"{SRC_DIR}/{PROTEIN_NAME}/ligand.mol2"
srcMolFile2 = f"{SRC_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_ligand.mol2"
destLigandPdbFile = f"{DEST_DIR}/{PROTEIN_NAME}/{PROTEIN_NAME}_ligand.pdb"

obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("mol2", "pdb")

if os.path.exists(srcMolFile1):
    obConversion.ReadFile(ligand, srcMolFile1)
elif os.path.exists(srcMolFile2):
    obConversion.ReadFile(ligand, srcMolFile2)
else:
    print("No ligand mol file found. Exiting.", file=sys.stderr)
    raise Exception("No input files found")
obConversion.WriteFile(ligand, destLigandPdbFile)



*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 1a1e_ligand)



True

In [8]:
# Select only chanins that are close to ligand
protein = prody.parsePDB(destPdbFile)
ligand = prody.parsePDB(destLigandPdbFile)
lresname = ligand.getResnames()[0]
complx = ligand + protein
complx = complx.select(f'same chain as exwithin 7 of resname {lresname}')

# Select only atoms that belongs to the protein
structure = complx.select(f'protein and not resname {lresname}')
prody.writePDB(destSelectedPdbFile, structure)
# pdb2pqr fails to read pdbs with the one line header generated by ProDy...
with open(destSelectedPdbFile, 'r') as fin:
    data = fin.read().splitlines(True)
with open(destSelectedPdbFile, 'w') as fout:
    fout.writelines(data[1:])

selected = prody.parsePDB(destSelectedPdbFile)   



@> 3235 atoms and 1 coordinate set(s) were parsed in 0.06s.
@> 73 atoms and 1 coordinate set(s) were parsed in 0.00s.
@> 3302 atoms and 1 coordinate set(s) were parsed in 0.04s.


In [12]:
# Calculate packet and write ti to corresponding pocket file.

selected = prody.parsePDB(destSelectedPdbFile)
complx = ligand + selected
lresname = ligand.getResnames()[0]
pocket = complx.select(f'same residue as exwithin 4.5 of resname {lresname}')
prody.writePDB(destSelectedPdbFile, pocket)  



@> 180 atoms and 1 coordinate set(s) were parsed in 0.00s.


'/Users/tevfik/Sandbox/github/PHD/data/dest_data/1a1e/1a1e_selected.pdb'